In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
import sys
import warnings

sys.path.insert(1, '../config/')
import kepler_configs


warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [106]:
lahore_poi_160 = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_ind160.shp')
lahore_poi_160 = lahore_poi_160[lahore_poi_160.first_type.isin(['locality',
'sublocality_level_3',
'neighborhood',
'sublocality_level_1'])!=True]
lahore_poi_160 = lahore_poi_160.reset_index(drop=True)

In [109]:
lahore_vor = gpd.read_file('../data/vector/dev/LAHORE/gmaps_places/lahore_voronoi_fishnet_1000m_updated.shp')

In [110]:
lahore_poi_160 = gpd.sjoin(lahore_vor,lahore_poi_160,how='right',op='contains')
lahore_poi_160 = lahore_poi_160.drop(columns=['index_left','geometry'])

In [113]:
lahore_poi_160.to_csv('../data/processed/lahore_unagg_160vor.csv',index=False)

In [76]:
lahore_poi_160 = lahore_poi_160[['vor_id','area_sq','area_sqrt','sum_pop','mean_pop','max_pop','min_pop','place_id',
                                 'user_ratin','rating','price_leve','first_type','second_typ']]

In [77]:
lahore_poi_160['occurances'] = 1
lahore_poi_160 = lahore_poi_160.drop(columns=['place_id'])

In [58]:
# pivot_test = lahore_poi_160[['vor_id','first_type','dummy_val']]
# pivot_test = pd.pivot_table(pivot_test,index=['vor_id'],columns=['first_type'],fill_value=0).reset_index()
# pivot_test.columns = pivot_test.columns.get_level_values(1)
# pivot_test.rename(columns={'':'vor_id'})

In [99]:
pivoted_lahore_poi = pd.pivot_table(lahore_poi_160,values=['user_ratin','rating','price_leve','occurances'],
               index=['vor_id','area_sq','area_sqrt','sum_pop','mean_pop','max_pop','min_pop'],
               columns=['first_type'],aggfunc={'occurances':np.sum,
                                              'user_ratin':np.sum,
                                              'rating':np.mean,
                                              'price_leve':np.mean})

In [100]:
pivoted_lahore_poi.columns = pivoted_lahore_poi.columns.map('-'.join).str.strip('-')
pivoted_lahore_poi = pivoted_lahore_poi.reset_index()
pivoted_lahore_poi.to_csv('../data/processed/lahore_agg_160vor.csv',index=False)